# Modeling
Pada tahapan ini, kita akan melakukan modeling menggunakan algoritma machine learning berdasarkan hasil RFM Analysis sebelumnya. Tujuan dari tahapan ini adalah untuk meningkatkan efektivitas dan efisiensi hasil yang telah didapatkan sebelumnya. Dengan menerapkan algoritma machine learning yang sesuai, diharapkan dapat menghasilkan segmentasi pelanggan yang lebih akurat dan dapat memberikan insight yang lebih baik untuk strategi bisnis ke depan.

In [ ]:
# Import Libraries yang diperlukan
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from ipywidgets import interact, Dropdown

from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
df = pd.read_csv('3. feature_engineering.csv')
rfm_df = pd.read_csv('3. rfm_df.csv')

# feature = '/content/drive/MyDrive/Final Project Experiment/3. feature_engineering.csv'
# df_rfm = '/content/drive/MyDrive/Final Project Experiment/3. rfm_df.csv'

# df = pd.read_csv(feature)
# rfm_df = pd.read_csv(df_rfm)

In [ ]:
df.head()

In [ ]:
rfm_df

## Data Preprocessing
Sebelum membangun model machine learning, perlu dilakukan tahap Data Preprocessing. Dalam konteks unsupervised learning, jarak antar data sangat berpengaruh terhadap model yang dihasilkan. Oleh karena itu, perlu dilakukan scaling pada fitur-fitur yang akan digunakan agar memiliki skala yang sama dan tidak mempengaruhi hasil model.

In [ ]:
scaler = MinMaxScaler()

# Menerapkan MinMax pada R_rank_boxcox, F_rank_boxcox, dan M_rank_boxcox
rfm_df['R_rank_scaled'] = scaler.fit_transform(rfm_df[['R_rank_boxcox']])
rfm_df['F_rank_scaled'] = scaler.fit_transform(rfm_df[['F_rank_boxcox']])
rfm_df['M_rank_scaled'] = scaler.fit_transform(rfm_df[['M_rank_boxcox']])

Karena seperti yang kita ketahui dalam penerapan nya K-Means sangat berpengaruh terhadap jarak, jadi kita akan melakukan scaling terhadap variable ranking karena nilai dari masing masing variable masih berbeda. Jika diperhatikan variable monetary memiliki rentang yang lebih besar dibandingkan nilai Recency dan Frequency. Sehingga kita perlu melakukan normalisasi skala kembali dengan scaler untuk membantu dalam analisa terkait pengaruh dari masing masing variable.

MinMaxScaler adalah salah satu metode penskalaan yang umum digunakan dalam preprocessing data. Metode ini mengubah skala data ke dalam rentang yang ditentukan, seringkali antara 0 dan 1. Metode ini bekerja dengan mengurangi nilai minimum dari setiap fitur dan kemudian membaginya dengan jangkauan (nilai maksimum dikurangi nilai minimum) dari fitur tersebut.

Pemilihan MinMaxScaler dapat memiliki beberapa alasan, di antaranya:

- Rentang yang terbatas: Dengan mengubah skala data ke rentang 0 hingga 1, semua fitur akan memiliki rentang yang seragam, yang dapat menghindari dominasi fitur dengan rentang yang lebih besar dalam proses pemodelan atau analisis selanjutnya.

- Perawatan outlier: MinMaxScaler secara default sensitif terhadap outlier. Jika terdapat outlier dalam data, rentang data yang dihasilkan akan menjadi lebih besar. Hal ini dapat bermanfaat jika outlier tersebut memiliki informasi penting atau merupakan kasus yang menarik untuk dianalisis.

**Namun kalau diperhatikan sebelumnya variabel F belum terdistribusi normal secara sempurna, mengapa tetap memakai MinMax?**

Jika diperhatikan data kita pada F_rank terdiri dari `1, 2, 3, 4, 5, 6, 11`.

Dalam contoh ini, kita memiliki nilai 11 yang dapat dianggap sebagai outlier. Namun, dampak outlier ini pada rentang dan perbandingan relatif antara nilai-nilai lainnya tidak terlalu signifikan.

Jika kita menggunakan metode penskalaan seperti MinMaxScaler pada contoh ini, rentang data akan tetap di antara 0 dan 1. Meskipun outlier memiliki perbedaan yang cukup besar dengan nilai-nilai lainnya (lebih dari dua kali lipat), rentang data yang dihasilkan masih dapat mempertahankan perbandingan relatif antara nilai-nilai lainnya.

Dalam hal ini, outlier nilai 11 dapat dianggap sebagai outlier yang tidak signifikan. Meskipun perbedaannya cukup besar dengan nilai-nilai lainnya, outlier ini tidak secara signifikan mengganggu analisis atau kesimpulan yang dapat ditarik dari dataset.

**Penjelasan lebih lanjutnya sebagai berikut:**

Dalam konteks ini, kita memiliki data sebagai berikut: `1, 2, 3, 4, 5, 6, 11`. Jika kita menggunakan MinMaxScaler untuk mentransformasikan data ini ke dalam rentang 0 hingga 1, kita akan mendapatkan data yang disesuaikan sebagai berikut: `0, 0.1, 0.2, 0.3, 0.4, 0.5, 1`.

Meskipun nilai 11 merupakan outlier yang jauh lebih tinggi daripada nilai-nilai lainnya, rentang data yang dihasilkan masih dapat mempertahankan perbandingan relatif antara nilai-nilai yang bukan outlier. Nilai-nilai 1 hingga 6 tetap memiliki perbedaan yang seragam di antara mereka setelah penskalaan, dan nilai-nilai tersebut tetap terjaga dalam urutan aslinya.

In [ ]:
rfm_df.head()

In [ ]:
rfm_df_norm = rfm_df[['R_rank_scaled', 'F_rank_scaled', 'M_rank_scaled']].copy()

In [ ]:
rfm_df_norm.head()

# K-Means
KMeans merupakan salah satu algoritma dalam machine learning yang digunakan untuk melakukan clustering atau pengelompokan data berdasarkan kesamaan fitur-fitur yang dimiliki oleh data tersebut. Algoritma ini bekerja dengan cara membagi data ke dalam beberapa cluster atau kelompok berdasarkan jarak antara data dengan pusat cluster (centroid) terdekat. KMeans menggunakan metode unsupervised learning, sehingga tidak memerlukan label atau target untuk melakukan clustering.

Pada dasarnya, semakin banyak segmentasi yang digunakan pada RFM analysis, semakin spesifik dan detail segmentasi yang dihasilkan. Karena RFM Analysis terdiri dari 3 buah fitur yang berbeda, maka pada project kali ini kita akan melakukan segmentasi setidaknya lebih dari 3 buah segmentasi untuk memudahkan identifikasi tiap segmen dan agar tidak bias / miss interpret. Dalam beberapa kasus, 4 segmentasi mungkin sudah cukup untuk mengidentifikasi pola perilaku pelanggan. Namun, terkadang 4 segmentasi saja tidak cukup untuk memberikan wawasan yang cukup tentang perilaku pelanggan, terutama jika bisnis memiliki banyak produk atau layanan, variasi harga yang berbeda, atau pasar yang sangat kompetitif.

Dengan menggunakan lebih dari 4 segmentasi, bisnis dapat mendapatkan wawasan yang lebih rinci tentang perilaku pelanggan, seperti preferensi produk yang lebih spesifik, kecenderungan pembelian pada periode tertentu, atau segmentasi pasar yang lebih tepat sasaran. Dengan demikian, bisnis dapat membuat keputusan yang lebih baik berdasarkan data dan memperbaiki strategi pemasaran dan penjualan mereka. Namun, harus diingat bahwa terlalu banyak segmentasi dapat menghasilkan kelompok yang terlalu kecil dan tidak mewakili secara signifikan populasi pelanggan secara keseluruhan.

In [ ]:
wcss = []
for i in range(3,16):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(rfm_df_norm)
    wcss.append(kmeans.inertia_)

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(3,16)), y=wcss, mode='lines+markers'))
fig.update_layout(title='Elbow Method for Optimal k',
                  xaxis_title='Number of Clusters (k)',
                  yaxis_title='WCSS')
pio.show(fig)

Jika diperhatikan pada grafik elbow method diatas. `Namun apa sih sebenarnya Elbow Method ini?`  

Jadi Elbow Method merupakan salah satu metode yang digunakan untuk menentukan banyaknya cluster yang optimal dalam algoritma K-Means. Berikut adalah langkah langkah dalam menentukan jumlah cluster yang optimal:

1. Melakukan clustering dengan dengan berbagai jumlah cluster, dalah hal ini kita memulai dari jumlah cluster yakni 3 buah seperti yang sudah dijelaskan sebelumnya.

2. Hitung nilai WCSS (Within-Cluster Sum of Square) atau bisa disebut sebagai nilai galat kuadrat dari setiap cluster.

3. WCSS biasanya terletak pada sumbu y sedangkan Jumlah cluster pada sumbu x.

4. Temukan sudut yang cenderung membentuk siku atau yang menunjukkan di mana penurunan WCSS mulai menurun secara lambat. Pada titik ini, penambahan jumlah cluster tidak lagi memberikan penurunan WCSS yang signifikan.

**Berdasarkan data yang berikan, berikut adalah analisa dari elbow method:**

- Jumlah kluster: 3, WCSS: 3.844
- Jumlah kluster: 4, WCSS: 2.720
- Jumlah kluster: 5, WCSS: 2.049
- Jumlah kluster: 6, WCSS: 1.689
- Jumlah kluster: 7, WCSS: 1.417
- Jumlah kluster: 8, WCSS: 1.223
- Jumlah kluster: 9, WCSS: 1.096
- Jumlah kluster: 10, WCSS: 0.991

Dalam elbow method, kita mencari penurunan yang signifikan dalam WCSS saat jumlah kluster meningkat. Tujuan kita adalah menemukan jumlah kluster yang memberikan penurunan yang signifikan dalam WCSS, tetapi setelah titik tersebut, penurunan tidak lagi signifikan secara substansial.

Dalam kasus ini, grafik WCSS menunjukkan penurunan yang cukup tajam dari 3 kluster hingga 4 kluster, dan penurunan yang lebih lambat dari 4 kluster hingga 6 kluster. Setelah 6 kluster, penurunan WCSS tidak lagi signifikan secara substansial.

Berdasarkan analisa ini, titik "siku" atau "elbow" mungkin berada `di antara 4 dan 6 kluster`. Namun, keputusan akhir tentang jumlah kluster yang optimal harus dipertimbangkan dengan faktor lain seperti interpretasi bisnis dan tujuan analisis dan perlu dicatat bahwa penggunaan metode Elbow Method terkadang dapat bersifat subjektif, sehingga diperlukan validasi menggunakan metode lain, seperti silhouette scores.

In [ ]:
silhouette_scores = []

for k in range(4, 11):
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
    kmeans.fit(rfm_df_norm)
    labels = kmeans.labels_
    silhouette_scores.append(silhouette_score(rfm_df_norm, labels))

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(4,11), y=silhouette_scores, mode='lines+markers', name='Silhouette Score'))
fig.update_layout(title='Silhouette Score vs Number of Clusters',
xaxis_title='Number of Clusters (k)', yaxis_title='Silhouette Score')
fig.show()

Silhouette score adalah metode evaluasi clustering yang digunakan untuk menentukan seberapa baik suatu data dikelompokkan dalam suatu cluster. Silhouette score menghitung seberapa dekat setiap titik data dalam suatu cluster dengan titik data di cluster lain dibandingkan dengan titik data di cluster yang sama.

Silhouette score berkisar antara -1 hingga 1, di mana nilai positif menunjukkan seberapa baik data dikelompokkan dan nilai negatif menunjukkan bahwa data seharusnya lebih baik dikelompokkan ke dalam cluster yang berbeda. Semakin dekat nilai Silhouette score dengan 1, semakin baik data dikelompokkan.


**Berdasarkan nilai silhouette score yang Anda berikan, berikut adalah kesimpulan yang dapat diambil:**

- Jumlah kluster: 4, Silhouette score: 0.374
- Jumlah kluster: 5, Silhouette score: 0.368
- Jumlah kluster: 6, Silhouette score: 0.369


Berdasarkan kesimpulan dari metode elbow dan silhouette score, Kita akan mempertimbangkan menggunakan 4 kluster atau 5 kluster atau 6 kluster sebagai jumlah kluster yang optimal. Namun, keputusan akhir harus dipertimbangkan dengan tujuan bisnis dan interpretasi hasil clustering yang dihasilkan. Maka selanjutnya kita akan coba untuk mebandingkan kedua jumlah kluster tersebut.

### 4 Clustering

In [ ]:
# melakukan k-means clustering dengan 4 cluster
kmeans = KMeans(n_clusters=4, init='k-means++', random_state=42)
kmeans.fit(rfm_df_norm)
labels_4 = kmeans.labels_

# menambahkan kolom labels hasil clustering ke dalam dataframe rfm_df
rfm_df['cluster_4'] = labels_4

# membuat scatter plot
fig = px.scatter_3d(rfm_df, x='R_rank_scaled', y='F_rank_scaled', z='M_rank_scaled', color='cluster_4',
                    symbol='cluster_4', opacity=0.8, width=800, height=600)

# menambahkan title pada plot
fig.update_layout(title_text="RFM Clustering")

# mengatur posisi dan tampilan legend
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    title='Cluster',
    orientation="h"))

# menampilkan plot
fig.show()

In [ ]:
# Buat dataframe dengan pivot table
pivot_table = rfm_df.pivot_table(index='cluster_4', values=['Recency', 'Frequency', 'Monetary'], aggfunc='mean') # 

# gradasi warna merah
cmap = plt.cm.get_cmap('Reds')

# Tambahkan parameter menunjukkan tinggi rendahnya angka dalam gradasi warna merah
pivot_table.style.background_gradient(cmap=cmap).format('{:.2f}')

**Analisis Segmentasi:**

- Terlihat bahwa pengelompokkan yang dilakukan terlalu general. Jumlah cluster yang terlalu sedikit dapat mengakibatkan pengelompokkan yang kurang representatif terhadap variasi data yang sebenarnya.
- Jika diperhatikan cakupan pada cluster 1 yang merupakan cluster memiliki recency sudah cukup lama, namun sebenarnya cluster ini masih bisa dibagi lagi menjadi 2 cluster yang lebih menggambarkan pembagian segmentasi nya.
- Karena pembagiannya terlalu general kami memutuskan untuk tidak melanjutkan analisa lebih lagi.

### 5 Clustering

In [ ]:
# melakukan k-means clustering dengan 5 cluster
kmeans = KMeans(n_clusters=5, init='k-means++', random_state=42)
kmeans.fit(rfm_df_norm)
labels_5 = kmeans.labels_

# menambahkan kolom labels hasil clustering ke dalam dataframe rfm_df
rfm_df['cluster_5'] = labels_5

# membuat scatter plot
fig = px.scatter_3d(rfm_df, x='R_rank_scaled', y='F_rank_scaled', z='M_rank_scaled', color='cluster_5',
                    symbol='cluster_5', opacity=0.8, width=800, height=600)

# menambahkan title pada plot
fig.update_layout(title_text="RFM Clustering")

# mengatur posisi dan tampilan legend
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    orientation="h"))

# menampilkan plot
fig.show()

**Analisa Segmentasi:**


Untuk menentukan apakah 5 klaster sudah cukup detail dan representatif terhadap data, ada beberapa faktor yang perlu dipertimbangkan:

- Jumlah cluster: Jumlah cluster yang terlalu sedikit dapat menghasilkan pengelompokkan yang terlalu umum dan tidak memberikan detail yang cukup. Sebaliknya, terlalu banyak cluster dapat menyulitkan interpretasi dan menghasilkan cluster yang tidak signifikan. Pemilihan 5 klaster mungkin telah mempertimbangkan keseimbangan antara detail dan interpretasi yang baik.

- Kesesuaian dengan tujuan analisis: Penting untuk memastikan bahwa klaster yang terbentuk sesuai dengan tujuan analisis dan pemahaman yang ingin dicapai. Jika tujuan analisis adalah untuk mengidentifikasi pola pembelian atau perilaku pelanggan, maka klaster harus mampu menjelaskan perbedaan dalam pola tersebut.

- Visualisasi dan pemahaman pola: Melalui visualisasi dan pemahaman pola atau perilaku yang berbeda-beda dari pelanggan dalam masing-masing klaster, Anda dapat membuat keputusan tentang apakah klaster tersebut sudah memberikan informasi yang cukup detail dan representatif terhadap data.

Coba kita lihat perbandingan dengan menggunakan 6 buah kluster.

**Analisa Lebih Lanjut 5 Buah Cluster:**

Berdasarkan informasi yang disajikan sebelumnya, terdapat beberapa temuan terkait klaster yang terbentuk:

- Cluster 0 memiliki proporsi terbanyak, dengan jumlah pelanggan yang baru dan cenderung memiliki pengeluaran yang signifikan. Cluster ini mungkin mencakup sebagian besar pelanggan baru yang belum terlalu aktif dalam bertransaksi.

- Cluster 2 memiliki proporsi terendah dengan hanya 1298 pelanggan. Klaster ini mungkin mencakup pelanggan yang sangat loyal dan memiliki tingkat pengeluaran yang tinggi. Pelanggan dalam klaster ini dapat dianggap sebagai kelompok yang paling aktif dan terlibat dalam bertransaksi di platform.

Karakteristik ini dapat memberikan wawasan yang berharga terkait segmentasi pelanggan. Dalam hal ini, pengelompokkan klaster dapat memberikan gambaran yang lebih rinci tentang perilaku dan preferensi pelanggan yang berbeda.

**Recency Analysis**

In [ ]:
# buat bins berdasarkan quantile
bins = pd.qcut(rfm_df.Recency, q=4, labels=['New', 'Active','Risk', 'Inactive'])

rfm_df['Recency_Bins'] = bins

cluster_rec = pd.pivot_table(rfm_df, values='customer_unique_id', index='cluster_5', columns='Recency_Bins', aggfunc='count', fill_value=0) 

cluster_rec

In [ ]:
i = ['New', 'Active','Risk', 'Inactive']
result = pd.DataFrame()
for label in i:
    temp = rfm_df[rfm_df['Recency_Bins']==label]['Recency'].describe()
    temp.name = label
    result = pd.concat([result, temp], axis=1)
result

**NOTE**:  
* `NEW: < 3 Bulan` 
* `Active: 4 < 6 Bulan`  
* `Risk: 7 < 9 Bulan`  
* `Inactive: > 12 Bulan`  

Berdasarkan data yang diberikan pada tabel di atas, kita dapat membuat analisa singkat sebagai berikut:

1. `Cluster 0`: Cluster ini memiliki proporsi yang cukup merata antara pelanggan dalam kategori `Inactive` dan `Risk`. Hal ini menunjukkan bahwa pelanggan dalam cluster ini cenderung tidak aktif dalam bertransaksi dalam kurun waktu 4 hingga 9 bulan terakhir.

2. `Cluster 1`: Cluster ini memiliki jumlah pelanggan yang relatif kecil dalam setiap kategori, termasuk "New", "Active", `New`, dan `Active`. Hal ini menunjukkan bahwa pelanggan dalam cluster ini memiliki tingkat aktivitas yang lebih tinggi secara keseluruhan.

3. `Cluster 2`: Cluster ini tidak memiliki pelanggan dalam kategori `inactive`, namun memiliki jumlah yang signifikan dalam kategori "Risk" dan "Inactive". Hal ini menunjukkan bahwa pelanggan dalam cluster ini memiliki risiko churn (berhenti bertransaksi) yang tinggi atau sudah tidak aktif dalam bertransaksi dalam kurun waktu 7 hingga 12 bulan terakhir.

4. `Cluster 3`: Cluster ini tidak memiliki pelanggan dalam kategori `Inactive`, namun memiliki jumlah yang signifikan dalam kategori `New` dan `Active`. Hal ini menunjukkan bahwa pelanggan dalam cluster ini didominasi pengguna baru.

5. `Cluster 4`: Cluster ini cukup seragam dari segi recency


In [ ]:
cluster_freq = pd.pivot_table(rfm_df, values='customer_unique_id', index='cluster_5', columns='Frequency', aggfunc='count', fill_value=0)

cluster_freq

**INSIGHT**:

Analisa ini memberikan gambaran singkat tentang pola frekuensi pembelian pada masing-masing klaster. Cluster 2 memiliki variasi frekuensi pembelian yang lebih tinggi dibandingkan dengan klaster lainnya. Cluster 0, 1, 3, dan 4 memiliki mayoritas pelanggan dengan frekuensi pembelian sebanyak 1.

In [ ]:
# buat bins berdasarkan quantile
bins = pd.qcut(rfm_df.Monetary, q=4, labels=['low', 'medium', 'high', 'very_high'])

rfm_df['Monetary_Bins'] = bins

cluster_mon = pd.pivot_table(rfm_df, values='customer_unique_id', index='cluster_5', columns='Monetary_Bins', aggfunc='count', fill_value=0)

cluster_mon

In [ ]:
i = ['low', 'medium', 'high', 'very_high']
result = pd.DataFrame()
for label in i:
    temp = rfm_df[rfm_df['Monetary_Bins']==label]['Monetary'].describe()
    temp.name = label
    result = pd.concat([result, temp], axis=1)
result

**INSIGHT**:

Analisa ini memberikan gambaran singkat tentang tingkat pengeluaran pelanggan dalam masing-masing klaster. Cluster 0 dan 1 memiliki mayoritas pelanggan dengan nilai Monetary yang tinggi, sedangkan Cluster 3 dan 4 memiliki mayoritas pelanggan dengan nilai Monetary yang rendah. Cluster 2 memiliki variasi dalam tingkat pengeluaran pelanggan.

In [ ]:
# Buat dataframe dengan pivot table
pivot_table = rfm_df.pivot_table(index='cluster_5', values=['Recency', 'Frequency', 'Monetary'], aggfunc='mean') # 

# Definisikan gradasi warna merah
cmap = plt.cm.get_cmap('Reds')

# Tambahkan parameter samping untuk menunjukkan tinggi rendahnya angka dalam gradasi warna merah
pivot_table.style.background_gradient(cmap=cmap).format('{:.2f}')

| Nama Cluster | Arti Cluster | Deskripsi Cluster |
|:--------------:|:--------------:|:---------------------:|
|Cluster 0 | Sleeping Giant | Cluster ini merupakan cluster yang dahulunya merupakan Big Spender, namun saat ini mereka sudah tidak aktif lagi berbelanja. Berdasarkan Recency nya mereka sudah tidak beraktifitas lebih dari 7 bulan, lalu pelanggan ini juga memiliki frekuensi hanya 1 kali  serta cluster ini memiliki tingkat monetary yang besar yaitu dengan rata-rata R262.50 yang merupakan tingkat monetary terbesar ke-2 dari seluruh cluster. Sehingga dengan demikian untuk kita perlu memperhatikan mereka agar tetap kembali aktif.|
|Cluster 1 | Newcomers | Karakteristik pelanggan yang baru melakukan pembelian pertama dengan mayoritas recency (0-6 bulan) namun mayoritas data berada pada range 0-3 bulan. Pelanggan ini memiliki frekuensi mayoritas sebanyak 1 kali serta Pelanggan ini memiliki tingkat pengeluaran kecil dengan rata-rata R59.79. Cluster ini memiliki jumlah data terbanyak ke-2 setelah cluster big spender dengan total pelanggan 16k orang. Sehingga dengan demikian cluster ini perlu diperhatikan karena dapat berpotensi menjadi pelanggan setia di masa depan.|
|Cluster 2 | Lost Customer | Cluster pelanggan ini merupakan pelanggan yang sudah lama tidak melakukan pembelian dengan dominasi range recency lebih dari 12 bulan. Cluster ini juga memiliki frekuensi hanya 1 kali selain itu pelanggan ini juga memiliki tingkat monetary yang cenderung kecil yaitu dengan rata-rata R63.61. Sehingga Perlu dicoba untuk mengaktifkan kembali minat mereka agar tetap menjadi pelanggan setia Anda.|
|Cluster 3 | Big Spenders | Pelanggan ini memiliki karakteristik baru melakukan pembelian pertama-nya dengan range recency (0-9 bulan) dengan dominasi pelanggan di range 0-6 bulan. Secara frekuensi pelanggan ini memiliki memiliki pelanggan yang mempunyai frekuensi hanya 1 kali, serta pelanggan ini memiliki rata-rata pengeluaran yang cenderung menengah-atas R238.12. Sehingga dengan demikian cluster ini memiliki potensi menjadi pelanggan setia dengan pengeluaran besar di masa depan. |
|Cluster 4 | Loyalist | Cluster ini dapat dikategorikan sebagai cluster terbaik serta loyal, karena berdasarkan Recency di dominasi pelanggan yang beraktifitas dalam kurun waktu 0-7 bulan. Pelanggan ini memiliki frekuensi lebih dari 1 kali terbanyak serta pelanggan ini memiliki rata-rata pengeluaran yang besar yaitu sebesar R276.09. Walaupun cluster ini sudah menjadi yang terbaik namun kita tetap harus menjaga mereka untuk menjadi pelanggan setia. |


Untuk menentukan apakah 5 klaster sudah cukup detail dan representatif terhadap data, terdapat beberapa hal yang perlu dipertimbangkan, seperti:

- Jumlah cluster: Apabila jumlah cluster terlalu sedikit maka pembentukan cluster akan menjadi general atau kurang detail. Jika diperlukan informasi yang lebih detail atau kompleks, jumlah cluster yang terbentuk dapat ditingkatkan. Namun, jumlah cluster yang terlalu banyak dapat menyulitkan interpretasi dan menghasilkan cluster yang tidak informatif atau tidak signifikan.

- Kesesuaian dengan tujuan analisis: Cluster yang terbentuk harus relevan dengan tujuan analisis dan pemahaman domain yang ingin dicapai. Misalnya, jika tujuan analisis adalah untuk menemukan pola pembelian atau perilaku pelanggan, maka cluster yang terbentuk harus mampu menjelaskan perbedaan dalam pola pembelian atau perilaku.

Dengan menggunakan ke-5 cluster ini, secara visualisasi dan pola atau perilaku dari pelanggan yang berbeda-beda kami memutuskan untuk menggunakan jumlah cluster tersebut.

`Namun bagaimana dengan jumlah 6 buah cluster apakah lebih baik? Mari kita cek..`

### 6 Clustering

In [ ]:
# melakukan k-means clustering dengan 6 cluster
kmeans = KMeans(n_clusters=6, init='k-means++', random_state=42)
kmeans.fit(rfm_df_norm)
labels_6 = kmeans.labels_

# menambahkan kolom labels hasil clustering ke dalam dataframe rfm_df
rfm_df['cluster_6'] = labels_6

# membuat scatter plot
fig = px.scatter_3d(rfm_df, x='R_rank_scaled', y='F_rank_scaled', z='M_rank_scaled', color='cluster_6',
                    symbol='cluster_6', opacity=0.8, width=800, height=600)

# menambahkan title pada plot
fig.update_layout(title_text="RFM Clustering")

# mengatur posisi dan tampilan legend
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    orientation="h"))

# menampilkan plot
fig.show()

In [ ]:
cluster_count = rfm_df.groupby('cluster_6')['customer_unique_id'].count()

fig = go.Figure(data=[
    go.Bar(x=cluster_count.index, y=cluster_count.values),
    go.Bar(x=cluster_count.index, y=cluster_count.values / cluster_count.sum(), visible=False)
])

fig.update_layout(
    title="Customer Count per Cluster",
    xaxis_title="Cluster ID",
    yaxis_title="Count",
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="Customer Count", method="update", args=[{"visible": [True, False]}]),
                dict(label="Normalized Customer Count", method="update", args=[{"visible": [False, True]}])
            ]),
        )
    ]
)

fig.update_traces(texttemplate='%{y}', textposition='outside') # Menambahkan anotasi pada masing-masing bar

fig.show()

Jika dibandingkan dengan 5 buah kluster, dengan menggunakan 6 buah kluster terlihat lebih detail. terutama pada bagian segmentasi customer yang memiliki tingkat pengeluaran menengah-rendah yang terbagi menjadi 3 buah kluster yang berbeda berdasarkan recency nya. 

Hal ini bisa membantu dalam membuat analisa yang lebih baik lagi, mengingat hasil visualisasi masih mendukung dari tujuan analisis kita.

**Analisa Lebih Lanjut 6 Cluster**

In [ ]:
# buat bins berdasarkan quantile
bins = pd.qcut(rfm_df.Recency, q=4, labels=['New', 'Active','Risk', 'Inactive'])

rfm_df['Recency_Bins'] = bins

cluster_rec = pd.pivot_table(rfm_df, values='customer_unique_id', index='cluster_6', columns='Recency_Bins', aggfunc='count', fill_value=0)

cluster_rec

**NOTE**:  
* `NEW: <= 3 Bulan` 
* `Active: 4 <= 6 Bulan`  
* `Risk: 7 <= 9 Bulan`  
* `Inactive: > 12 Bulan`  

Berdasarkan data yang diberikan pada tabel di atas, kita dapat membuat analisa singkat sebagai berikut:

1. `Cluster 0`: Cluster ini memiliki proporsi yang cukup merata antara pelanggan dalam kategori `Inactive` dan `Risk`. Hal ini menunjukkan bahwa pelanggan dalam cluster ini cenderung tidak aktif dalam bertransaksi dalam kurun waktu 4 hingga 9 bulan terakhir.

2. `Cluster 1`: Cluster ini memiliki dominasi pada kategori `New` dan `Active`, sehingga dapat dikatakan kategori ini merupakan pelanggan yang baru baru ini melakukan transaksi.

3. `Cluster 2`: Cluster ini memiliki pelanggan dalam kategori `Active` dan `Risk` dengan range terakhir kali aktifitas 4-9 bulan. Kelompok ini dapat dikatakan sebagai cluster churn risk.

4. `Cluster 3`: Cluster ini memiliki beragam kategori dari segi recency.

5. `Cluster 4`: Cluster ini dari segi recency memiliki dominasi pada kategori `New` dan `Active` sehingga dapat dikatakan juga sebagai pelanggan baru.

6. `Cluster 5` : Cluster ini memiliki dominasi pada kategori `Inactive`.


In [ ]:
i = ['New', 'Active','Risk', 'Inactive']
result = pd.DataFrame()
for label in i:
    temp = rfm_df[rfm_df['Recency_Bins']==label]['Recency'].describe()
    temp.name = label
    result = pd.concat([result, temp], axis=1)
result

In [ ]:
cluster_freq = pd.pivot_table(rfm_df, values='customer_unique_id', index='cluster_6', columns='Frequency', aggfunc='count', fill_value=0)

cluster_freq

**Insight**: 

Dari segi frekuensi seluruh segmentasi kecuali cluster 3 tidak memiliki pelanggan yang melakukan transaksi lebih dari 1 kali. Hal ini dapat terjadi karena seperti yang sudah diketahui bahwa secara natural dataset ini memang memiliki masalah dalam mempertahankan pelanggan.

In [ ]:
# buat bins berdasarkan quantile
bins = pd.qcut(rfm_df.Monetary, q=4, labels=['low', 'medium', 'high', 'very_high'])

rfm_df['Monetary_Bins'] = bins

cluster_mon = pd.pivot_table(rfm_df, values='customer_unique_id', index='cluster_6', columns='Monetary_Bins', aggfunc='count', fill_value=0)

cluster_mon

In [ ]:
i = ['low', 'medium', 'high', 'very_high']
result = pd.DataFrame()
for label in i:
    temp = rfm_df[rfm_df['Monetary_Bins']==label]['Monetary'].describe()
    temp.name = label
    result = pd.concat([result, temp], axis=1)
result

**Insight**:

- `Cluster 0 & 4` cenderung memiliki karakteristik pengeluaran yang serupa yaitu di dominasi oleh pelanggan yang menghabiskan banyak uang.

- `Cluster 1, 2 & 5` dalam segi monetary memiliki karakteristik yang serupa yaitu di dominasi oleh pelanggan yang sedikit menghabiskan uangnya.

- `Cluster 3` dalam segi monetary cukup beragam namun memiliki dominasi pada pelanggan yang menghabiskan banyak uang.

In [ ]:
# Buat dataframe dengan pivot table
pivot_table = rfm_df.pivot_table(index='cluster_6', values=['Recency', 'Frequency', 'Monetary'], aggfunc='mean') # 

# Definisikan gradasi warna merah
cmap = plt.cm.get_cmap('Reds')

# Tambahkan parameter samping untuk menunjukkan tinggi rendahnya angka dalam gradasi warna merah
pivot_table.style.background_gradient(cmap=cmap).format('{:.2f}')

| Nama Cluster | Arti Cluster | Deskripsi Cluster |
|:--------------:|:--------------:|:---------------------:|
|Cluster 0 | Sleeping Giant | Cluster ini merupakan cluster yang dahulunya merupakan Big Spender, namun saat ini mereka sudah tidak aktif lagi berbelanja. Berdasarkan Recency nya mereka sudah tidak beraktifitas lebih dari 7 bulan, lalu pelanggan ini juga memiliki frekuensi hanya 1 kali  serta cluster ini memiliki tingkat monetary yang besar yaitu dengan rata-rata R200.43 yang merupakan tingkat monetary terbesar ke-1 dari seluruh cluster. Sehingga dengan demikian untuk kita perlu memperhatikan mereka agar tetap kembali aktif.|
|Cluster 1 | Newcomers | Karakteristik pelanggan yang baru melakukan pembelian pertama dengan mayoritas recency (0-6 bulan) namun mayoritas data berada pada range 0-3 bulan. Pelanggan ini memiliki frekuensi mayoritas sebanyak 1 kali serta Pelanggan ini memiliki tingkat pengeluaran kecil dengan rata-rata R59.99. Cluster ini memiliki jumlah data terbanyak ke-2 setelah cluster big spender dengan total pelanggan 16k orang. Sehingga dengan demikian cluster ini perlu diperhatikan karena dapat berpotensi menjadi pelanggan setia di masa depan.|
|Cluster 2 | Churn Risk | Cluster pelanggan ini merupakan pelanggan yang tidak terlalu lama  sekitar 4-9 bulan dengan dominasi pada range 7-9 bulan. Cluster ini hanya memiliki 1 kali frekuensi dengan rata rata pengeluaran sebesar R66.20 yang termasuk rendah dibanding cluster lainnya. Sehingga perlu di coba untuk meretain mereka agar tidak menjadi Lost Customer. |
|Cluster 3 | Loyalist | Cluster ini dapat dikategorikan sebagai cluster terbaik serta loyal, karena berdasarkan Recency di dominasi pelanggan yang beraktifitas dalam kurun waktu 0-7 bulan. Pelanggan ini memiliki frekuensi lebih dari 1 kali terbanyak serta pelanggan ini memiliki rata-rata pengeluaran yang besar yaitu sebesar R241.79. Walaupun cluster ini sudah menjadi yang terbaik namun kita tetap harus menjaga mereka untuk menjadi pelanggan setia. |
|Cluster 4 | Big Spenders | Pelanggan ini memiliki karakteristik baru melakukan pembelian pertama-nya dengan range recency (0-9 bulan) dengan dominasi pelanggan di range 0-6 bulan. Secara frekuensi pelanggan ini memiliki memiliki pelanggan yang mempunyai frekuensi hanya 1 kali, serta pelanggan ini memiliki rata-rata pengeluaran yang cenderung menengah-atas R189.97. Sehingga dengan demikian cluster ini memiliki potensi menjadi pelanggan setia dengan pengeluaran besar di masa depan. |
|Cluster 5 | Lost Customer | Cluster pelanggan ini merupakan pelanggan yang sudah lama tidak melakukan pembelian dengan dominasi range recency lebih dari 12 bulan. Cluster ini juga memiliki frekuensi hanya 1 kali selain itu pelanggan ini juga memiliki tingkat monetary yang cenderung kecil yaitu dengan rata-rata R65.18. Sehingga Perlu dicoba untuk mengaktifkan kembali minat mereka agar tetap menjadi pelanggan setia Anda.|


Setelah melakukan analisis lebih lanjut, kami menemukan bahwa menggunakan 6 buah cluster memberikan hasil yang lebih baik dibandingkan dengan 5 buah cluster. Keputusan ini didasarkan pada visualisasi yang lebih mudah dipahami serta pembentukan cluster yang lebih detail dan jelas. Oleh karena itu, kami memutuskan untuk melanjutkan analisis dengan menggunakan 6 buah cluster guna mendapatkan pemahaman yang lebih mendalam mengenai karakteristik pelanggan.

In [ ]:
# Mengubah penamaan cluster menjadi lebih interpretable
rfm_df['cluster_name'] = np.where(rfm_df['cluster_6'] == 0, 'Sleeping Giant',
                            np.where(rfm_df['cluster_6'] == 1, 'Newcomers',
                                    np.where(rfm_df['cluster_6'] == 2, 'Churn Risk',
                                            np.where(rfm_df['cluster_6'] == 3, 'Loyalist',
                                                     np.where(rfm_df['cluster_6'] == 4, 'Big Spenders', 'Lost Customer')
                                                    ))))

# Segmentasi Analysis

Setelah melakukan pengelompokkan cluster berdasarkan RFM Analysis, langkah selanjutnya adalah melakukan analisis terhadap setiap segmentasi yang terbentuk dengan menganalisisnya menggunakan fitur lain. Dengan demikian, akan didapatkan informasi yang lebih lengkap mengenai perilaku dan karakteristik dari setiap segmen pelanggan. Hal ini akan membantu dalam pengambilan keputusan strategis, seperti mengembangkan produk, meningkatkan loyalitas pelanggan, atau meningkatkan efektivitas pemasaran untuk setiap segmen pelanggan.

In [ ]:
# Membuat cluster yang nantinya akan digabungkan dengan df
cluster = rfm_df[['customer_unique_id', 'Recency', 'Frequency', 'Monetary',  'cluster_6', 'cluster_name']].copy()
# cluster.rename(columns={'cluster_6': 'cluster'}, inplace=True)

In [ ]:
# Menggabungkan hasil pembentukan cluster dengan dataframe df untuk dilakukan analisa kembali
df = df.merge(cluster, how='left', on='customer_unique_id')
df = df.drop(columns='Total')

In [ ]:
# membuat variabel recency_month
df['Recency_Month'] = df['Recency'] // 30

In [ ]:
# membuat variabel recency_period
df['Recency_Period'] = np.where(df['Recency_Month'] <= 3, '<=3 Month',
                                np.where(df['Recency_Month'] <= 6, '4-6 Month',
                                        np.where(df['Recency_Month'] <= 9, '7-9 Month', '>12 Month')))

In [ ]:
# membuat variabel recency_period
df['Frequency_Period'] = np.where(df['Frequency'] == 1, '1 times',
                                np.where(df['Frequency'] == 2, '2 times',
                                        np.where(df['Frequency'] == 3, '3 times',
                                                np.where(df['Frequency'] == 4, '4 times', '>=5 times'))))

In [ ]:
# Membuat variabel lifespan
df["order_approved_at"] = pd.to_datetime(df['order_approved_at'])
latest_date = df["order_approved_at"].max()

df["Lifespan"] = latest_date - df.groupby("customer_unique_id")["order_approved_at"].transform("max")

df["Lifespan"] = df["Lifespan"].dt.days

df['Lifespan_Month'] = df['Lifespan'] // 30

df['Lifespan_Period'] = np.where(df['Lifespan_Month'] <= 3, '<=3 Month',
                                np.where(df['Lifespan_Month'] <= 6, '4-6 Month',
                                        np.where(df['Lifespan_Month'] <= 9, '7-9 Month', '>12 Month')))

In [ ]:
pt = df.pivot_table(index='Category', columns='cluster_name', values='order_id', aggfunc='count', fill_value=0)

# list warna
colors = ['rgba(50, 171, 96, 0.6)', 'rgba(128, 0, 128, 0.6)', 'rgba(255, 69, 0, 0.6)', 
          'rgba(255, 215, 0, 0.6)', 'rgba(220, 20, 60, 0.6)', 'rgba(0, 191, 255, 0.6)', 
          'rgba(0, 0, 205, 0.6)', 'rgba(255, 99, 71, 0.6)']

# Menghitung rata-rata jumlah untuk setiap kategori
avg_values = pt.mean(axis=1)

fig = go.Figure()

for i, col in enumerate(pt.columns):
    fig.add_trace(
        go.Bar(
            x=pt.index,
            y=pt[col],
            name=col,
            text=pt[col],
            textposition='auto',
            marker=dict(
                color=colors[i],
                line=dict(
                    color='rgba(50, 171, 96, 1.0)',
                    width=1
                )
            )
        )
    )

# Menambahkan garis rata-rata
fig.add_trace(
    go.Scatter(
        x=pt.index,
        y=avg_values,
        mode='lines',
        name='Average',
        line=dict(
            color='black',
            width=2,
            dash='dash'
        )
    )
)

fig.update_layout(
    title='Pivot Table Barplot with Average Line',
    xaxis_tickangle=-45,
    xaxis_title='Category',
    yaxis_title='Count',
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig.show()

**Insight**:

- Jika diperhatikan tiap cluster memiliki pola yang hampir sama untuk penjualan di tiap kategori-nya, dimana big spender selalu mendominasi penjualan hampir di tiap kategori kecuali pada kategori `clothing & apparel`, `food & beverages`, dan `Games / Entertainment` yang berurutan didominasi oleh churn risk, newcomers, dan sleeping giant.

- **Big Spenders**, memiliki minat yang besar terhadap `Electronics & Gadgets`, `Furniture`, `Health & Beauty`, `Sport Products`, hal ini terlihat dari jumlah pembeli yang jauh melebihi jauh dari rata-rata penjualan kategori tersebut.

- **Loyalist**, jumlah nya terlihat lebih rendah dibandingkan seluruh cluster lainnya dan jauh dibawah rata-rata penjualan tiap kategori tersebut. Untuk meningkatkan penjualan di tiap kategori nya kita dapat melakukannya dengan cara memberikan diskon / penawaran khusus untuk cluster ini.

- **Churn Risk**, memiliki minat yang tinggi juga pada kategori `furniture` serta pada kategori `clothing dan apparel` miliki penjualan tertinggi dibanding segmentasi lainnya, kerena potensi ini sehingga kita bisa membuat churn risk lebih tertarik lagi pada kategori tersebut. Selain itu kategori ini juga memiliki beberapa kategori yang dapat dimanfaatkan untuk mencegah mereka churn diantaranya `Electronics`, `Health and Beauty`, `Sports Products`.

- **Sleeping Giant**, terlihat dari pola memiliki tingkat penjualan hampir selalu di bawah rata rata penjualan di kategori nya, cluster ini memiliki keteratrikan yang cukup besar terhadap kategori `furniture` yang memiliki penjualan diatas rata-rata serta `Entertainment` hal ini bisa dilihat dari gap penjualan di kategori tersebut yang cukup tinggi.

- **Lost Customer**, terlihat memiliki penjualan yang cenderung di bawah rata rata per kategori nya, namun kluster ini memiliki penjualan tertinggi di `Furniture`.

- **Newcomers**, memiliki tren yang mendekati rata rata dari penjualan kategori, jika diperhatikan mereka memiliki tingkat penjualan tertinggi dibanding cluster lainnya pada kategori `food and beverages`, sehingga kita bisa memaksimalkan hal ini untuk terus meningkatkan penjualan di kategori tersebut, selain itu segmentasi customer ini juga memiliki potensi ketertarikan pada `furniture`, `electronics`, dan `health and beauty` hal ini terlihat dari tren penjualan nya yang sedikit diatas rata-rata.

In [ ]:
# membuat pivot tabel untuk menghitung tipe pembayaran berdasarkan cluster
pivot_table = pd.pivot_table(df, 
                             values='customer_unique_id', 
                             index=['cluster_name'], 
                             columns=['payment_type'], 
                             aggfunc='count', 
                             fill_value=0)

# menghitung proporsi setiap cluster
payment_type_props = pivot_table.apply(lambda x: x/x.sum(), axis=1)

payment_type_props = payment_type_props.reset_index()

# Membuat grafik menggunakan plotly
payment_type_props_melted = payment_type_props.melt(id_vars=['cluster_name'], 
                                                   var_name='payment_type', 
                                                   value_name='prop')
fig = px.bar(payment_type_props_melted, 
             x="cluster_name", 
             y="prop", 
             color="payment_type",
             barmode='stack')
fig.update_layout(title='Payment Types for Each Cluster Name',
                  xaxis_title='Cluster Name',
                  yaxis_title='Proportion of Payment Types')
fig.show()

Dari sini terlihat bahwa dari seluruh segmentasi memiliki kesamaan dalam hal tipe pembayaran. Metode Pembayaran yang dominan digunakan oleh customers adalah `credit_card` lalu diikuti oleh `boleto (pembayaran khusus)` lalu diikuti oleh penggunaan voucher dan terakhir yaitu `debit_card`.

Berdasarkan temuan tersebut, satu solusi yang bisa diterapkan untuk mempertahankan retensi pelanggan adalah dengan memberikan insentif khusus pada penggunaan metode pembayaran tertentu, terutama credit_card dan boleto. Hal ini bisa dilakukan dengan memberikan `potongan harga` atau `reward points` yang lebih tinggi untuk transaksi yang menggunakan metode pembayaran tersebut. Selain itu, perusahaan juga bisa mempertimbangkan untuk `memperluas pilihan metode pembayaran` yang tersedia, sehingga pelanggan memiliki lebih banyak pilihan dan bisa menggunakan metode pembayaran yang paling nyaman untuk mereka. Dengan cara ini, diharapkan dapat meningkatkan loyalitas pelanggan dan mempertahankan retensi pelanggan.

In [ ]:
fig = px.box(df, x="cluster_name", y="price", color="cluster_name")
fig.show()

**Insight**:

Sleeping Giant, Big Spenders dan Loyalist memiliki ketertarikan terhadap barang-barang yang memiliki harga cenderung tinggi jika dibandingkan dengan 3 cluster lainnya.

In [ ]:
# pivot tabel untuk menghitung recency period berdasarkan cluster
pivot_table = pd.pivot_table(df, values='customer_unique_id', index='cluster_name', columns='Recency_Period', aggfunc='count', fill_value=0)

# normalize proporsi
prop_table = pivot_table.div(pivot_table.sum(axis=1), axis=0)
prop_table = prop_table[['<=3 Month', '4-6 Month', '7-9 Month', '>12 Month']]

# membuat grafik
fig = go.Figure(data=[go.Bar(x=prop_table.columns, y=prop_table.loc[cluster_name], name=cluster_name) for cluster_name in prop_table.index])
fig.update_layout(barmode='stack', xaxis_title='Recency Period', yaxis_title='Proportion', title='Recency Period for Each Cluster Name')
fig.show()

**Insight**:

- Proporsi pengguna yang sudah tidak aktif yang paling banyak adalah yang lebih dari 12 bulan.
- Loyalist memiliki range pembelian yang cukup variatif, namun tetap dimoniasi oleh mereka yang tetap aktif dalam kurun waktu 3 bulan terakhir.
- Sleeping Giant memiliki proporsi yang cukup besar, sehingga akan lebih baik apabila kita dapat mempertahankan segmentasi customer tersebut.
- Untuk meningkatkan retensi pelanggan, dapat dilakukan strategi pemasaran khusus untuk kelompok ini, seperti memberikan penawaran khusus atau diskon untuk mendorong mereka untuk melakukan pembelian kembali. Selain itu, perlu dilakukan analisis lebih lanjut untuk memahami penyebab dari "sleeping giants" dan "lost customers" serta mencari solusi yang tepat untuk menarik kembali ke aktifitas pembelian.

In [ ]:
# Membuat pivot tabel dengan frekuensi
pivot_table = pd.pivot_table(df, values='customer_unique_id', index='Frequency_Period', columns='cluster_name', aggfunc='count', fill_value=0)

# normalize proporsi
prop_table = pivot_table.div(pivot_table.sum(axis=0), axis=1)

# Membuat stack barplot
fig = go.Figure(data=[go.Bar(x=prop_table.columns, y=prop_table.loc[cluster_name], name=cluster_name) for cluster_name in prop_table.index])
fig.update_layout(barmode='stack', xaxis_title='Frequency Period', yaxis_title='Proportion', title='Frequency Period for Each Cluster Name')
fig.show()

**Insight**:

Dari sini terlihat bahwa memang permasalahan di Olist Brazillian E-Commerce adalah dalam menangani retensi dari customers. Berdasarkan informasi diatas terlihat bahwa customer dominan melakukan transaksi hanya 1 kali saja. Oleh karena itu, perusahaan perlu memperkuat strategi retensi pelanggan dan memikirkan cara untuk membuat pelanggan menjadi lebih loyal pada bisnis mereka.

In [ ]:
# Membuat pivot tabel untuk order_day
pivot_table = pd.pivot_table(df, values='customer_unique_id', index='cluster_name', columns='order_day', aggfunc='count', fill_value=0)

# normalize proporsi
prop_table = pivot_table.div(pivot_table.sum(axis=1), axis=0)
prop_table = prop_table[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]

# membuat stacked barplot
fig = go.Figure(data=[go.Bar(x=prop_table.columns, y=prop_table.loc[cluster_name], name=cluster_name) for cluster_name in prop_table.index])
fig.update_layout(barmode='stack', xaxis_title='Order Day', yaxis_title='Proportion', title='Order Day for Each Cluster Name')
fig.show()

**Insight**:

- Terjadi penurunan yang tidak terlalu signifikan pada Weekdays

- Weekend cenderung lebih rendah dibandingkan Weekdays

- Kita dapat memanfaatkan momen Weekend untuk meningkatkan penjualan, sehingga kita perlu membuat sebuah campaign yang berlaku hanya pada weekend untuk meningkatkan penjualan di weekend.


In [ ]:
# Membuat grafik penjualan mingguan
df_copy = df.copy()

df_copy['order_approved_at'] = pd.to_datetime(df_copy['order_approved_at'])
df_copy['Month'] = df_copy['order_approved_at'].dt.month
df_copy['Week'] = pd.qcut(df_copy['order_approved_at'].dt.day, q=4, labels=False)

monthly_sales = df_copy.groupby(['Month', 'Week'])['payment_value'].sum().reset_index()

@interact
def plot_sales_by_month(month=Dropdown(options=df_copy['Month'].unique())):
    month_data = monthly_sales[monthly_sales['Month'] == month]
    
    fig = px.bar(month_data, x='Week', y='payment_value',
                 labels={'payment_value': 'Penjualan', 'Week': 'Minggu'},
                 title=f'Penjualan Mingguan Bulan {month}')
    
    fig.update_layout(xaxis=dict(tickmode='array', tickvals=[0, 1, 2, 3], ticktext=['Minggu 1', 'Minggu 2', 'Minggu 3', 'Minggu 4']))
    
    fig.show()

In [ ]:
df_copy = df.copy()

df_copy['order_approved_at'] = pd.to_datetime(df_copy['order_approved_at'])
df_copy['Month'] = df_copy['order_approved_at'].dt.month
df_copy['Week'] = pd.qcut(df_copy['order_approved_at'].dt.day, q=4, labels=False)

monthly_sales = df_copy.groupby(['Month', 'Week', 'cluster_name'])['payment_value'].sum().reset_index()

@interact
def plot_sales_by_month(month=Dropdown(options=df_copy['Month'].unique()), cluster=Dropdown(options=df_copy['cluster_name'].unique())):
    month_data = monthly_sales[(monthly_sales['Month'] == month) & (monthly_sales['cluster_name'] == cluster)]
    
    fig = px.bar(month_data, x='Week', y='payment_value',
                 labels={'payment_value': 'Penjualan', 'Week': 'Minggu'},
                 title=f'Penjualan Mingguan Bulan {month} (Cluster {cluster})')
    
    fig.update_layout(xaxis=dict(tickmode='array', tickvals=[0, 1, 2, 3], ticktext=['Minggu 1', 'Minggu 2', 'Minggu 3', 'Minggu 4']))
    
    fig.show()


**Insight**:

- Penjualan tertinggi pada Pekan-1 terjadi pada bulan `Februari`, `May`, `Juni`, `Agustus` dan `Desember`.
- Penjualan tertinggi pada Pekan-3 terjadi pada bulan `Januari`, `Maret`, `September`, dan `Oktober`
- Penjualan tertinggi pada Pekan-4 terjadi pada bulan `Juli` dan `November`
- Penjualan pada Pekan-2 belum pernah mencapai puncak jika dibandingkan dengan pekan lainnya
- Untuk meningkatkan sales kita dapat menggunakan beberapa strategi diantaranya adalah memberikan `flash-sale` yang terjadi pada Pekan-1 karena berdasarkan data Pekan-1 terjadi puncak terbanyak, atau kita menerapkan nya pada pekan-2 dengan tujuan meningkatkan salses pada pekan-2 tiap bulannya.

In [ ]:
# buat bins berdasarkan quantile
bins = pd.qcut(df.delivery_time_days, q=4, labels=['quick', 'normal', 'slow', '?'])

df['delivery_time_days_bins'] = bins

delivery_time_days_bins = pd.pivot_table(df, values='customer_unique_id', index='cluster_name', columns='delivery_time_days_bins', aggfunc='count', fill_value=0)

delivery_time_days_bins

In [ ]:
i = ['quick', 'normal', 'slow', '?']
result = pd.DataFrame()
for label in i:
    temp = df[df['delivery_time_days_bins']==label]['delivery_time_days'].describe()
    temp.name = label
    result = pd.concat([result, temp], axis=1)
result

Secara keseluruhan, penting untuk memperhatikan waktu pengiriman dan memastikan pengiriman yang cepat dan konsisten untuk meningkatkan retensi pelanggan.

Dari informasi diatas kita perlu melakukan evaluasi terkait kebijakan pengiriman barang terutama yang dapat dikendalikan oleh sellers. Seperti memberikan jaminan atau garansi sampai dalam waktu tertentu. Kebijakan seperti ini dapat membantu meningkatkan pada sektor `Food and Beverages` untuk memastikan kualitas dari makanan itu sendiri.

In [ ]:
# save to csv
df.to_csv('cluster_6.csv', index=False)

# Recommendation

Berdasarkan hasil analisa dan informasi yang telah diperoleh sebelumnya kita memutuskan untuk menghasilkan 2 buah rekomendasi dalam scope yang berbeda yaitu secara general dan berdasarkan hasil analisa dengan RFM Analisis. Scope rekomendasi secara general diperoleh berdsarkan informasi yang diperoleh dari EDA dan domain knowledge terkait, sedangkan Scope rekomendasi untuk hasil segmentasi diperoleh berdasarkan analisa dengan RFM dan juga K-Means.



**Secara general**, seperti yang kita sudah ketahui bahwa Olist merupakan sebuah E-Commerce baru yang membutuhkan beberapa strategi marketing yang tepat. Secara General, terdapat beberapa faktor yang dapat dipertimbangkan oleh olist:

- **Lakukan sosialisasi dan promosi** yang intensif di daerah-daerah yang memiliki sedikit penjual untuk menarik minat para penjual. Fokuskan upaya pemasaran untuk mengedukasi penjual potensial tentang manfaat dan keuntungan bergabung dengan Olist. 

- Oleh karena lamanya proses pengiriman barang yang bervariasi dan terkadang terlalu lama, kami merekomendasikan **implementasi program garansi atau jaminan barang sampai tepat waktu**. Dengan adanya program garansi atau jaminan barang sampai tepat waktu, diharapkan Olist dapat meningkatkan kepercayaan pelanggan, meminimalisir keterlambatan pengiriman, dan memberikan pengalaman yang lebih baik bagi pelanggan.

- Tingkatkan fitur layanan untuk **kebutuhan sehari-hari** serta implementasikan **Program langganan autodebet** akan memberikan kemudahan kepada pelanggan dengan menghilangkan kebutuhan untuk melakukan pembayaran manual setiap bulan.

- Untuk meningkatkan penjualan dan menarik perhatian pelanggan, kami merekomendasikan implementasi **program diskon khusus** yang mirip dengan *Black Friday* setiap bulan, namun dengan penekanan khusus pada **akhir pekan** di pekan-1 tiap bulannya.

- Terapkan Flash Sale, dan berikan promo yang sesuai dengan personalisasi pengguna selain itu terus tingkatkan **awareness** terhadap kategori yang sangat kecil penjualannya misal *Food and Beverages*.

- Kita dapat lihat bahwa food and beverages merupakan catogery terendah, sebagai langkah awal, kita dapat memberikan rekomendasi kepada team marketing untuk **drop item** tersebut, karena item tersebut tidak begitu baik dalam segi penjualan, di sisi lain, item2 tersebut merupakan item2 yg berbeda dari pada item2 lain dalam hal bahwa *food and beverages* merupakan item yg memiliki kadarluasa (memerlukan waktu yg cepat utk jika tidak ingin menanggung loss)

- Kita dapat memberika insentif seperti item bundle jika membeli furniture untuk memanfaatkan tinggi nya penjualan furniture, dan juga kita dapat memberikan promosi buy 1 get 1 atau pun diskon kepada item2 lain2 yg kurang baik performanya dalam segi penjualan seperti Clothing & Apparel.

- Jika budget masih memungkinkan, kita dapat memberikan insentif reward kepada customer2 yg membeli furniture untuk memperkuat marketing strategi utk category furniture.

- Untuk tim data warehouse perlu melakukan validasi data kembali untuk meminimalisir kerusakan data, serta untuk tim data scientist dan analyst perlu cross check terlebih dahulu sebelum menggunakan data.

- Untuk variasi model kita bisa menggunakan model DBSCAN, PCA, Gaussian, etc.

Sedangkan berikut adalah **rekomendasi strategi** yang dapat diterapkan Perusahaan untuk **meningkatkan Retensi yang juga akan meningkatkan Sales** berdasarkan hasil temuan dari Analisa RFM dengan menggunakan K-Means:

| Nama Cluster | Deskripsi |
|:--------------:|:--------------:|
| Loyalist | Berikan Reward pada program loyalitas, Sosialisasikan terkait program prioritas sehingga dengan menjadi customers akan selalu di prioritaskan dengan memberikan layanan yang cepat, Berikan program referensi yang terhubung dengan program loyalitas, lebih banyak tawarkan promo pada kategori yang rendah pembeli seperti food and beverage / stationary / dll, tawarkan promo program autodebet untuk pembayaran kebutuhan harian  |
| Big Spenders | Berikan promo bundle pada barang barang yang mereka minati, berikan potongan diskon dengan syarat minimum pengeluaran, berikan promo ongkos kirim dengan kuota pemakaian, sosialisasikan manfaat dari program loyalitas, tawarkan promo program autodebet untuk pembayaran kebutuhan harian |
| Newcomers | Berikan potongan harga / promo khusus untuk cluster ini dengan adanya syarat minimum pengeluaran, Sosialisasikan bahwa terdapat program loyalitas dan bagi pengguna baru perlu mengumpulkan poin untuk unlock program tersebut, berikan potongan ongkos kirim dengan kuota pemakaian |
| Churn Risk Buyers | Minta survei penggunaan, tetap jaga komunikasi dengan menawarkan promo promo sesuai dengan barang yang diminati dengan adanya batasan pengeluaran, berikan ongkos kirim untuk, berikan penwaran khusus, sosialisasikan bagaimana cara mengaktifkan program loyalitas |
| Sleeping Giant |menghubungi customer terkait untuk mensosialisasikan program loyalitas, berikan promo yang menarik terkait kategori yang paling diminati agar mereka kembali aktif, berikan reward gratis ongkir untuk pembelian selanjutnya, tawarkan promo program autodebet untuk pembayaran kebutuhan harian|
| Lost Customers |menghubungi customer terkait untuk mensosialisasikan adanya program loyalitas, kirimkan notifikasi secara berkala melalui email, minta feedback melalui email, jangan terlalu mengeluarkan dana yang telalu besar untuk menarik mereka kembali |

**Simulasi Penerapan Segmentasi**

Untuk membuktikan apakah hasil modeling yang kita lakukan ini searah dengan salah satu tujuan yang ingin kita capai yaitu meminimalisir biaya strategi marketing:

* Tanpa Model (Kita akan menghubungi customer - customer untuk menarik kembali customer agar berbelanja di Olist E-commerce):
    - Total cost (Telepon, Email, etc) => 100 BRL
    - Total Customer => 10.000 orang
    - Total Customer on target (Lost Customer or Sleeping Giant) => 6.000 orang
    - Total Customer off target (Loyalist, Big Spenders, New Comers, & Churn Risk) => 4.000 orang
    - Total Loss due to off target => 100 BRL * 4.000 Person = **-400.000 BRL**
        
Jika kita tidak menggunakan model untuk menghubungi pelanggan-pelanggan yang menjadi target agar berbelanja di Olist E-commerce, perusahaan akan mengalami kerugian sebesar -400.000 BRL karena pelanggan-pelanggan yang bukan target terkena program tersebut.
    
* Dengan Model:
    - Total cost (specific to category furniture) => 100 BRL
    - Total Customer on target (Lost Customer or Sleeping Giant) => 6.000 orang
    - Total Loss due to off target => 100 BRL * 0 Person = **0 BRL**

Jika kita tidak menggunakan model untuk menghubungi customer - customer untuk menarik kembali customer agar berbelanja di Olist E-commerce maka perusahaan akan kehilangan (Loss) sebesar **-400.000 BRL** karena terdapat customer - customer off target, jika kita menggunakan model customer -customer yang off target tidak akan terkena program tersebut sehingga loss dapat dihindari.

**Overall**:
Dengan menggunakan model, kita dapat memfokuskan upaya pemasaran hanya pada pelanggan yang memiliki potensi tinggi untuk berbelanja kembali, yang dapat membantu mengurangi biaya pemasaran secara keseluruhan dan memaksimalkan efektivitas upaya pemasaran.

# Conclusion

Berdasarkan hasil yang telah kita lakukan maka dapat kita dapat menyimpulkan menjadi 2 bagian yaitu secara model dan bisnis:


**Kesimpulan berdasarkan Model**  
1. Meskipun Olist memiliki banyak pelanggan baru setiap bulan, tingkat retensi pelanggan sangat rendah dengan hanya sekitar 1% pelanggan yang menggunakan layanan Olist setiap bulannya.

2. **Pembagian Cluster**: Berdasarkan analisis RFM, kita melakukan pembagian pelanggan ke dalam beberapa cluster. Cluster ini mencerminkan karakteristik dan perilaku pelanggan yang berbeda.
    - `*Big Spenders*`: Pelanggan yang melakukan pembelian dengan nilai tertinggi (Recency rendah, Frequency tinggi, Monetary Value tinggi).
    - `*Sleeping Giant*`: Pelanggan yang belum melakukan pembelian baru-baru ini (Recency tinggi), tetapi memiliki nilai pembelian yang tinggi di masa lalu (Frequency tinggi, Monetary Value tinggi).
    - `*Loyalist*`: Pelanggan yang secara konsisten melakukan pembelian (Recency rendah, Frequency tinggi, Monetary Value cenderung tinggi).
    - `*Newcomers*`: Pelanggan baru yang baru saja melakukan pembelian (Recency rendah, Monetary Rendah, Frequency rendah).
    - `*Lost Customers*`: Pelanggan yang sudah lama tidak melakukan pembelian (Recency tinggi) dan memiliki nilai pembelian rendah (Frequency rendah, Monetary Value rendah).
    - `*Churn Risk*`: Pelanggan yang berisiko beralih atau tidak melakukan pembelian lagi (Recency tinggi, Frequency rendah, Monetary Value rendah).

3. **Metode Evaluasi Cluster**: Untuk memilih jumlah cluster yang optimal, kita menggunakan metode silhouette score. Silhouette score memberikan gambaran seberapa baik pemisahan antara cluster dan seberapa homogen data dalam setiap cluster.

4. **Hasil Evaluasi**: Meskipun cluster terbaik berdasarkan silhouette score adalah 4 cluster dengan score 0.374, kita memilih menggunakan 6 cluster dengan score 0.369. Ini dikarenakan pembagian dengan 5-6 cluster memberikan tingkat detail yang lebih baik daripada 4 cluster yang terlalu umum.

5. **Hasil Rekomendasi**: Berdasarkan hasil dari RFM Analisis dan K-Means kita dapat menentukan rekomendasi yang tepat untuk melakukan strategi marketing, dan diharapkan dari terbentuknya segmentasi ini dapat meminimalisir loss yang terjadi pada biaya strategi marketing, serta diharapkan dengan rekomendasi yang diberikan mampu untuk meningkatkan retensi serta sales dari Olist.


**Kesimpulan berdasarkan Bisnis**
1. Terdapat banyak customer baru tiap bulan, tetapi retention rate nya sangat tinggi (hanya sekitar 1% customer yang memakai olist tiap bulannya). (Berhubungan dengan tujuan)
2. Product terbaik dari Olist adalah Furniture, sementara product terburuk adalah Food & Beverages. (Kesimpulan utk di EDA)
3. Majoritas customer Olist menggunakan credit card dengan jangka waktu pembayaran adalah 1 bulan. (Kesimpulan utk di EDA)
4. State yang paling ramai transaksi adalah Sao Paulo, dengan total customer 20.000+ dan total seller 20.000+. (Kesimpulan utk di EDA) 
5. Sau Paulo menjadi state yang paling ramai bisa dikarenakan harga jual dan harga freight nya yang sangat terjangkau. Rata-rata dan median dari harga jual dan freight price di Sau Paulo adalah yang termurah dibandingkan dengan state lainnya. (Kesimpulan utk di EDA)